In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#export
from lib.nb_05 import *

### LM Data

In [4]:
path = datasets.untar_data(datasets.URLs.IMDB)

In [5]:
path

PosixPath('/Users/vks/.fastai/data/imdb')

In [6]:
path.ls()

[PosixPath('/Users/vks/.fastai/data/imdb/ld.pkl'),
 PosixPath('/Users/vks/.fastai/data/imdb/test'),
 PosixPath('/Users/vks/.fastai/data/imdb/ll_lm.pkl'),
 PosixPath('/Users/vks/.fastai/data/imdb/tmp_clas'),
 PosixPath('/Users/vks/.fastai/data/imdb/vocab_lm.pkl'),
 PosixPath('/Users/vks/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/Users/vks/.fastai/data/imdb/unsup'),
 PosixPath('/Users/vks/.fastai/data/imdb/README'),
 PosixPath('/Users/vks/.fastai/data/imdb/tmp_lm'),
 PosixPath('/Users/vks/.fastai/data/imdb/ll_clas.pkl'),
 PosixPath('/Users/vks/.fastai/data/imdb/train')]

In [7]:
il = TextList.from_files(path, extensions=".txt", include=['train', 'test', 'unsup'])

In [8]:
il

TextList (100000 items)
[PosixPath('/Users/vks/.fastai/data/imdb/test/neg/1821_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/9487_1.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/4604_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/2828_2.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/10890_1.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/3351_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/8070_2.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/1027_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/8248_3.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/4290_4.txt')...]
Path: /Users/vks/.fastai/data/imdb

In [9]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [10]:
sd

SplitData
Train: TextList (89786 items)
[PosixPath('/Users/vks/.fastai/data/imdb/test/neg/1821_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/9487_1.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/4604_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/2828_2.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/10890_1.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/3351_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/8070_2.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/1027_4.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/8248_3.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/4290_4.txt')...]
Path: /Users/vks/.fastai/data/imdb
Valid: TextList (10214 items)
[PosixPath('/Users/vks/.fastai/data/imdb/test/neg/6648_2.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/3725_1.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/4235_1.txt'), PosixPath('/Users/vks/.fastai/data/imdb/test/neg/11558_3.txt'), PosixP

In [11]:
proc_tok, proc_num =  TokenizeProcessor(max_workers=8), NumericalizeProcessor()

In [12]:
ll = label_by_func(sd, lambda y: 0, proc_x=[proc_tok, proc_num])

In [14]:
# pickle.dump(ll, open(path/'ll_lm.pkl', 'wb'))
# pickle.dump(proc_num.vocab, open(path/'vocab_lm.pkl', 'wb'))

In [17]:
ll = pickle.load(open(path/'ll_lm.pkl', 'rb'))
vocab = pickle.load(open(path/'vocab_lm.pkl', 'rb'))

In [18]:
bs,bptt = 64,70
data = lm_databunchify(ll, bs, bptt)

In [19]:
data.train_ds

In [20]:
proc_num.vocab = vocab

In [21]:
len(proc_num.vocab)

60003

### AWD-LSTM

[Regularizing and Optimizing LSTM Language Models](https://arxiv.org/abs/1708.02182)

Dataset for lanfuage modelling : https://github.com/alvations/Quotables/

This is important for the blog: https://elc.github.io/posts/embed-interactive-notebooks/

#### LSTM from Scratch

<img src="./images/LSTM.PNG" alt="LSTM" style="width:500px;">

https://forums.fast.ai/t/about-the-part-2-alumni-2018-category/13050/92

In [ ]:
class LSTMCell(nn.Module):
    
    def __init__(self, ni, nh):
        super().__init__()
        self.ih = nn.Linear(ni, 4*nh)
        self.hh = nn.Linear(nh, 4*nh)
        
    def forward(self, input, state):
        h,c = state
        #One big multiplication for all the gates is better than 4 smaller ones
        gates = (self.ih(input) + self.hh(h)).chunk(4, 1)
        input_gate, forget_gate, output_gate = map(torch.sigmoid, gates[:3])
        
        cell_gate = gates[3].tanh()
        c = (input_gate*cell_gate) + (forget_gate*c)
        h = output_gate*(c.tanh())
        return h, (h,c)

Then we create a LSTM Layer

In [ ]:
class LSTMLayer(nn.Module):
    
    def __init__(self, cell, *cell_args):
        super().__init__()
        self.cell = cell(*cell_args)
        
    def forward(self, input, state):
        inputs = input.unbind(1)
        outputs = []
        for i in range(len(inputs)):
            out, state = self.cell(inputs[i], state)
            outputs += [out]
        return torch.stack(outputs, dim=1), state

In [ ]:
lstm = LSTMLayer(LSTMCell, 300, 300)

In [ ]:
x = torch.randn(64, 70, 300)
h = (torch.zeros(64, 300),torch.zeros(64, 300))

In [ ]:
y,h1 = lstm(x,h)

In [ ]:
x.shape, h[0].shape, h[1].shape

In [ ]:
y.shape, h1[0].shape, h1[1].shape

In [ ]:
%timeit -n 10 y,h1 = lstm(x,h)

Builtin version in Pytorch.

In [ ]:
lstm = nn.LSTM(300, 300, 1, batch_first=True)

In [ ]:
x = torch.randn(64, 70, 300)
h = (torch.zeros(1, 64, 300),torch.zeros(1, 64, 300))

In [ ]:
%timeit -n 10 y,h1 = lstm(x,h)

In [ ]:
??nn.LSTM

#### Dropout

https://arxiv.org/pdf/1904.13310.pdf

We usually apply dropout by drawing a mask which tells us which elements to nullify. First, we'll need all different kinds of dropouts. Dropout consists into replacing some coefficients by 0 with probability p. To ensure that the average of the weights remains constant, we apply a correction to the weights that aren't nullified of a factor `1/(1-p)` (think of what happens to the activations if you want to figure out why!)

In [ ]:
x = torch.randn(100, 1000)
w = torch.randn(1000, 5)

In [ ]:
o = torch.matmul(x,w)

In [ ]:
o.shape, o.mean(), o.std()

In [22]:
#export
def dropout_mask(x, sz, p):
    return x.new(*sz).bernoulli_(1-p).div_(1-p)

In [ ]:
x = torch.randn(10,10)

In [ ]:
mask = dropout_mask(x, (10,10), 0.5); mask

Once with have a dropout mask `mask`, applying the dropout to `x` is simply done by `x = x * mask`. We create our own dropout mask and don't rely on pytorch dropout because we do not want to nullify all the coefficients randomly: on the sequence dimension, we will want to have always replace the same positions by zero along the seq_len dimension.

In [ ]:
(x*mask).std(),x.std()

EmbeddingDropout applies dropout to full rows of the embedding matrix.

In [23]:
#export
class EmbeddingDropout(nn.Module):
    
    def __init__(self, emb, emb_p):
        super().__init__()
        
        self.emb, self.emb_p = emb, emb_p
        
        self.pad_idx = self.emb.padding_idx
        if self.pad_idx is None: self.pad_idx = -1
            
    def forward(self, words, scale=None):
        
        if self.training and self.emb_p != 0:
            size = (self.emb.weight.size(0),1)
            mask = dropout_mask(self.emb.weight.data, size, self.emb_p)
            masked_embed = self.emb.weight * mask
        else: masked_embed = self.emb.weight
        
        if scale: masked_embed.mul_(scale)
        
        return F.embedding(words, masked_embed, self.pad_idx, self.emb.max_norm,
                           self.emb.norm_type, self.emb.scale_grad_by_freq, self.emb.sparse)

In [ ]:
enc = nn.Embedding(100, 7, padding_idx=1)
enc_dp = EmbeddingDropout(enc, 0.5)
tst_input = torch.randint(0,100,(8,))
enc_dp(tst_input)

`WeightDropout` is dropout applied to the weights of the inner LSTM hidden to hidden matrix. This is a little hacky if we want to preserve the CuDNN speed and not reimplement the cell from scratch. We add a parameter that will contain the raw weights, and we replace the weight matrix in the LSTM at the beginning of the forward pass.

In [24]:
#export

WEIGHT_HH = 'weight_hh_l0'

class WeightDropout(nn.Module):
    
    def __init__(self, module, weight_p=[0.], layer_names=[WEIGHT_HH]):
        super().__init__()
        
        self.module, self.weight_p, self.layer_names = module, weight_p, layer_names
        
        for layer in self.layer_names: 
            #Makes a copy of the weights of the selected layers.
            w = getattr(self.module, layer)
            #print(w.shape)
            self.register_parameter(f"{layer}_raw", nn.Parameter(w.data))
            self._parameters[layer] = F.dropout(w, p=self.weight_p, training=False)
            
    def _set_weights(self):
        for layer in self.layer_names:
            raw_w = getattr(self, f'{layer}_raw')
            self.module._parameters[layer] = F.dropout(raw_w, p=self.weight_p, training=self.training)
            
    def forward(self, *args):
        self._set_weights()
        with warnings.catch_warnings():
            #To avoid the warning that comes because the weights aren't flattened.
            warnings.simplefilter("ignore")
            return self.module.forward(*args)

In [ ]:
module = nn.LSTM(5, 2)
dp_module = WeightDropout(module, 0.7)
getattr(dp_module.module, WEIGHT_HH)

In [ ]:
tst_input = torch.randn(4,20,5)
h = (torch.zeros(1,20,2), torch.zeros(1,20,2))
x,h = dp_module(tst_input,h)
getattr(dp_module.module, WEIGHT_HH)

Inside a RNN, a tensor `x` will have three dimensions: `bs, seq_len, vocab_size`.  Recall that **we want to consistently apply the dropout mask across the seq_len dimension**, therefore, *we create a dropout mask for the first and third dimension and broadcast it to the seq_len dimension*.

In [25]:
#export
class RNNDropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p=p

    def forward(self, x):
        if not self.training or self.p == 0.: return x
        m = dropout_mask(x.data, (x.size(0), 1, x.size(2)), self.p)
        return x * m

In [ ]:
dp = RNNDropout(0.3)
tst_input = torch.randn(3,3,4)
tst_input, dp(tst_input)

In [26]:
#export
def to_detach(h):
    "Detaches `h` from its history."
    return h.detach() if type(h) == torch.Tensor else tuple(to_detach(v) for v in h)

In [27]:
#export
class AWD_LSTM(nn.Module):
    "AWD-LSTM inspired by https://arxiv.org/abs/1708.02182."

    initrange=0.1
    
    def __init__(self, vocab_sz, emb_sz, n_hid, n_layers, pad_token,
                hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5):
        
        super().__init__()
        
        self.bs, self.emb_sz, self.n_hid, self.n_layers = 1, emb_sz, n_hid, n_layers
        #Embedding and Embedding Dropout
        self.emb = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.emb_dp = EmbeddingDropout(self.emb, embed_p)
        self.emb.weight.data.uniform_(-self.initrange, self.initrange)
        
        ## RNN and Weight Dropout
        self.rnns = [nn.LSTM(self.emb_sz if l == 0 else self.n_hid,
                             (n_hid if l != n_layers - 1 else emb_sz),
                             batch_first=True) for l in range(self.n_layers)]
        self.rnns = nn.ModuleList([WeightDropout(rnn, weight_p) for rnn in self.rnns])
        
        ##Input anf HH Drpoouts
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(self.n_layers)])
        
    def forward(self, input):
        
        bs,sl = input.size()
        
        ## takes care of the last batch and the first, getting a new hidden state ?? 
        if bs!=self.bs:
            self.bs = bs 
            self.reset()
            
        raw_output = self.input_dp(self.emb_dp(input))
        
        new_hidden,raw_outputs,outputs = [],[],[]
        
        # we do this because we are not using built in LSTM n_layers
        # the reason being, we want to add dropout
        # does this introduce timing delays ?? 
        for l, (rnn, hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            raw_output, new_h = rnn(raw_output, self.hidden[l]) # where is this self.hidden coming from ? 
            new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = hid_dp(raw_output)
            outputs.append(raw_output) 
            
        self.hidden = to_detach(new_hidden)
        
        return raw_outputs, outputs
    
    def _one_hidden(self, l):
        "Return one hidden state."
        nh = self.n_hid if l != self.n_layers - 1 else self.emb_sz
        return next(self.parameters()).new(1, self.bs, nh).zero_()
    
    def reset(self):
        """Reset the hidden states"""
        self.hidden = [(self._one_hidden(l), self._one_hidden(l)) for l in range(self.n_layers)]

On top of this, we will apply a linear decoder. It's often best to use the same matrix as the one for the embeddings in the weights of the decoder.

In [28]:
#export
class LinearDecoder(nn.Module):
    
    def __init__(self, n_out, n_hid, output_p, tie_encoder=None, bias=True):
        super().__init__()
        
        self.output_dp = RNNDropout(output_p)
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight
        else: nn.init.kaiming_uniform_(self.decoder.weight)
            
    def forward(self, inputs):
        raw_outputs, outputs = inputs
        output = self.output_dp(outputs[-1]).contiguous()
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded, raw_outputs, outputs

In [29]:
#export
class SequentialRNN(nn.Sequential):
    "A sequential module that passes the reset call to its children."
    def reset(self):
        for c in self.children():
            if hasattr(c, 'reset'): c.reset()

In [30]:
#export
def get_awd_lstm_language_model(vocab_sz, emb_sz, n_hid, n_layers, pad_token, output_p=0.4, hidden_p=0.2, input_p=0.6, 
                       embed_p=0.1, weight_p=0.5, tie_weights=True, bias=True):
    rnn_enc = AWD_LSTM(vocab_sz, emb_sz, n_hid=n_hid, n_layers=n_layers, pad_token=pad_token,
                       hidden_p=hidden_p, input_p=input_p, embed_p=embed_p, weight_p=weight_p)
    enc = rnn_enc.emb if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(vocab_sz, emb_sz, output_p, tie_encoder=enc, bias=bias))

In [31]:
tok_pad = vocab.index(PAD)

In [ ]:
tok_pad

In [ ]:
tst_model = get_awd_lstm_language_model(len(vocab), 300, 300, 2, tok_pad)

In [ ]:
x,y = next(iter(data.train_dl))

In [ ]:
x.shape, y.shape

In [ ]:
z = tst_model(x)

In [ ]:
len(z)

In [ ]:
decoded, raw_outputs, outputs = z

The decoded tensor is flattened to `bs * seq_len` by `len(vocab)`:

In [ ]:
decoded.size()

In [ ]:
64*70

`raw_outputs` and `outputs` each contain the results of the intermediary layers:

In [ ]:
len(raw_outputs),len(outputs)

In [ ]:
[o.size() for o in raw_outputs], [o.size() for o in outputs]

### Callbacks to train a RNN 

First we add `CudaCallback` to utilize our `GPU's`

In [32]:
#export
class CudaCallback(Callback):
    def __init__(self,device): self.device=device
    def begin_fit(self): self.model.to(self.device)
    def begin_batch(self): self.run.xb,self.run.yb = self.xb.to(self.device),self.yb.to(self.device)

Now when we define our cbs we will call this callback using a partial with device set as below.
```python
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
```

We will clip the gradients. This is a classic technique that will allow us to use a higher learning rate by putting a maximum value on the norm of the gradients.

In [33]:
#export
class GradientClipping(Callback):
    
    def __init__(self, clip=None): self.clip = clip
    
    def after_backward(self):
        if self.clip: nn.utils.clip_grad_norm_(self.run.model.parameters(), self.clip)

Then we add an `RNNTrainer` that will do four things:
- change the output to make it contain only the `decoded` tensor (for the loss function) and store the `raw_outputs` and `outputs`
- apply Activation Regularization (AR): we add to the loss an L2 penalty on the last activations of the AWD LSTM (with dropout applied)
- apply Temporal Activation Regularization (TAR): we add to the loss an L2 penalty on the difference between two consecutive (in terms of words) raw outputs
- trigger the shuffle of the LMDataset at the beginning of each epoch

In [34]:
#export
class RNNTrainer(Callback):
    def __init__(self, α, β): self.α,self.β = α,β
    
    def after_pred(self):
        #Save the extra outputs for later and only returns the true output.
        self.raw_out,self.out = self.pred[1],self.pred[2]
        self.run.pred = self.pred[0]
    
    def after_loss(self):
        #AR and TAR
        if self.α != 0.:  self.run.loss += self.α * self.out[-1].float().pow(2).mean()
        if self.β != 0.:
            h = self.raw_out[-1]
            if len(h)>1: self.run.loss += self.β * (h[:,1:] - h[:,:-1]).float().pow(2).mean()
                
    def begin_epoch(self):
        #Shuffle the texts at the beginning of the epoch
        if hasattr(self.dl.dataset, "batchify"): self.dl.dataset.batchify()

In [35]:
#export
def cross_entropy_flat(input, target):
    bs,sl = target.size()
    return F.cross_entropy(input.view(bs * sl, -1), target.view(bs * sl))

def accuracy_flat(input, target):
    bs,sl = target.size()
    return accuracy(input.view(bs * sl, -1), target.view(bs * sl))

In [36]:
emb_sz, nh, nl = 300, 300, 2
model = get_awd_lstm_language_model(len(vocab), emb_sz, nh, nl, tok_pad, 
                                    input_p=0.6, output_p=0.4, weight_p=0.5, embed_p=0.1, hidden_p=0.2)

In [37]:
#export
def get_device():
    return torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [38]:
get_device()

device(type='cpu')

In [39]:
cbs = [partial(AvgStatsCallback,accuracy_flat),
       partial(CudaCallback, device=get_device()), 
       Recorder,
       partial(GradientClipping, clip=0.1),
       partial(RNNTrainer, α=2., β=1.),
       ProgressCallback]

In [40]:
learn = Learner(model, data, cross_entropy_flat, lr=5e-3, cb_funcs=cbs, opt_func=adam_opt())

In [ ]:
learn.fit(1)

In [41]:
!python notebook2script.py 06_awd_lstm.ipynb

Converted 06_awd_lstm.ipynb to lib/nb_06.py
